# Merge ontology sandbox
The purpose of this notebook is to boot up a Neuroglancer instance where the merge onotology tools are available for use. These are not implemented into Neuroglancer yet, so in the meantime this provides a sandbox for experimenting with these tools.

You will need to make sure to have the file allen.json downloaded. This is available in this git repository under the subfolder data/ from this notebook's location.

In [18]:
import neuroglancer
import json, csv
import numpy as np
import graphviz
import pandas as pd
%matplotlib inline  

In [19]:
df_filename = '/home/ahoag/ngdemo/data/allen_atlas/allen_id_table_w_voxel_counts_hierarch_labels.csv'
df_allen = pd.read_csv(df_filename)
ids = df_allen['reassigned_id']
names = df_allen['name']
ontology_id_dict = {ids[ii]:names[ii] for ii in range(len(df_allen))}
ontology_name_dict = {names[ii]:ids[ii] for ii in range(len(df_allen))}

In [20]:
ontology_file = 'data/allen.json'

with open(ontology_file) as json_file:
    data = json.load(json_file)

In [21]:
""" Make the graph that will be used
in the get_parent() function below """

def make_id_graph(dic,graph):
    """ Make a edge-unweighted directed graph from a dictionary
    Representing a brain ontology
    """
    name = dic.get('name')
    acronym = dic.get('acronym')
    children = dic.get('children')
    orig_id = dic.get('id')
    new_id = dic.get('graph_order') + 1
    graph.node(name,f'{acronym}: {new_id}')
    for child in children:
        child_name = child.get('name')
        graph.edge(name,child_name)
        make_id_graph(child,graph)
    return 

Gnew = graphviz.Digraph()
make_id_graph(data,Gnew)

In [22]:
""" helper graph functions """
def get_progeny(dic,input_nodename,progeny_list=None):
    """
    Gets all of the descendents of a given input nodename.
    --- INPUT ---
    dic             The dictionary representing the JSON ontology graph
    input_nodename   The name of the region whose progeny you want to know
    """
    if progeny_list == None:
        progeny_list = []
    if input_nodename == 'root':
        return list(ontology_name_dict.keys()) 
        
    name = dic.get('name')

    children = dic.get('children')
    if name == input_nodename:
        for child in children: # child is a dict
            child_name = child.get('name')
            progeny_list.append(child_name)
            get_progeny(child,input_nodename=child_name,progeny_list=progeny_list)
        return
    
    for child in children:
        child_name = child.get('name')
        get_progeny(child,input_nodename=input_nodename,progeny_list=progeny_list)
    return progeny_list

def get_parent(graph,input_nodename):
    if len(input_nodename.split(' ')) > 1:
        nodename_to_search = f'"{input_nodename}"'
    else:
        nodename_to_search = input_nodename
    edges_pointing_to_node=[x for x in graph.body if f'-> {nodename_to_search}' in x]
    if len(edges_pointing_to_node) == 0:
        return None
    elif len(edges_pointing_to_node) > 1:
        print("Error. There should not be more than one edge pointing to this node")
    else:
        parent_nodename = edges_pointing_to_node[0].split('->')[0].strip()
        # remove the extra quotes surrounding the nodename if there is more than one word
        if len(parent_nodename.split(' ')) > 1:
            return parent_nodename[1:-1]
        else:
            return parent_nodename
    return

In [23]:
def init_tool(s):
    print("in init_tool()")
    with viewer.config_state.txn() as st:
        try:
            del st.status_messages['hello']
        except KeyError:
            pass
    print("made it here")
    """ first figure out the selected layer """
    with viewer.txn() as txn:
        if len(txn.layers) > 1:
            selected_layer_name = txn.selectedLayer.layer
            if not selected_layer_name:
                with viewer.config_state.txn() as st:
                    st.status_messages['hello'] = 'No layer selected. Select a layer (right click or ctrl+click the layer panel)'        
                    return None, None
        elif len(txn.layers) == 1:
            print("should be here")
            selected_layer_name = txn.layers[0].name
        else:
            with viewer.config_state.txn() as st:
                st.status_messages['hello'] = 'No layers loaded. First load a layer to use this tool'        
                return None, None
    print("selecting region_map")
    print("layer name:")
    print(selected_layer_name)
    print("selected values:")
    print(s.selected_values)
    try:
        region_map = s.selected_values[selected_layer_name]
    except KeyError:
        # you need to move your cursor to get the layer to be selectable again
        return None, None
    named_tuple = region_map.value
    print("end of init_tool()")
    print("named tuple:")
    print(named_tuple)
    return named_tuple, selected_layer_name

def contract_atlas(s):
    named_tuple, selected_layer_name = init_tool(s)
    if not selected_layer_name:
        return
    if named_tuple:
        with viewer.config_state.txn() as st:
            st.status_messages['hello'] = 'key p pressed: contracting atlas' 
        if named_tuple.value:
            region_id = named_tuple.value
        else:
            region_id = named_tuple.key
        region_name = ontology_id_dict[region_id]
        # Look up parent name and then get corresponding ID
        parent_name = get_parent(Gnew,region_name)
        if not parent_name:
            with viewer.config_state.txn() as st:
                st.status_messages['hello'] = 'No parent found.'
            return
        parent_id = ontology_name_dict.get(parent_name)
        # find all progeny of this parent
        progeny_list = get_progeny(data,input_nodename=parent_name) # progeny names
        # initialize our equivalence list using the id-parent relationship we just found
        equivalence_list = [] 
        # Get the progeny ids and include them in the equivalence list
        for progeny_name in progeny_list:
            progeny_id = ontology_name_dict.get(progeny_name)
            if progeny_id:
                equivalence_list.append((progeny_id,parent_id)) 
        with viewer.txn() as txn:
            existing_equivalences = list(txn.layers[selected_layer_name].layer.equivalences.items())
            final_equivalence_list = existing_equivalences + equivalence_list
            txn.layers[selected_layer_name].layer.equivalences = final_equivalence_list
        return
    else:
        with viewer.config_state.txn() as st:
            st.status_messages['hello'] = 'No segment under cursor. Hover over segment to enable hierarchy tools' 
        return
    
def expand_atlas(s):
    named_tuple, selected_layer_name = init_tool(s)
    if not selected_layer_name:
        return
    
    if named_tuple:
        with viewer.config_state.txn() as st:
            st.status_messages['hello'] = 'key c pressed: expanding atlas' 
        """ if the hovered segment is mapped to a parent, then start at the parent level"""
        if named_tuple.value:
            region_id = named_tuple.value
        else:
            region_id = named_tuple.key
        """ Find and remove any existing equivalences that involve this region_id"""
        with viewer.txn() as txn:
            equiv_map = txn.layers[selected_layer_name].layer.equivalences
            if region_id not in equiv_map.keys():
                with viewer.config_state.txn() as st:
                    st.status_messages['hello'] = 'This segment is already at the lowest level in the hierarchy'
            else:
                equiv_map.delete_set(region_id)
        return
    else:
        with viewer.config_state.txn() as st:
            st.status_messages['hello'] = 'No segment under cursor. Hover over segment to enable hierarchy tools' 
        return

In [27]:
# key binding to merge segment with its parent segment using the ontology
def add_key_bindings():
    index = np.random.randint(0,100000)
    viewer.actions.add(f'contract-atlas{index}', contract_atlas)
    viewer.actions.add(f'expand-atlas{index}', expand_atlas)
    with viewer.config_state.txn() as s:
        s.input_event_bindings.viewer['keyp'] = f'contract-atlas{index}'
        s.input_event_bindings.viewer['keyc'] = f'expand-atlas{index}'
        s.status_messages['hello'] = 'Welcome to the merge ontology example. Press p to go up a level, c to go down a level.'


In [30]:
neuroglancer.set_static_content_source(url='http://localhost:8080')
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.layers['layer 1'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1340'
    )
#     s.layers['layer 2'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1337'
#     )
#     s.layers['layer 3'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1337'
#     )
print(viewer)

http://127.0.0.1:43781/v/d17476cc755272b880437d34c89d0c9d011ee536/


In [31]:
add_key_bindings()

in init_tool()
made it here
should be here
selecting region_map
layer name:
layer 1
selected values:
Map({"layer 1": {"value": {"key": "725", "label": "Isocortex-un7: Isocortex, unassigned area 7"}}})
end of init_tool()
named tuple:
SegmentIdMapEntry(key=725, value=None, label='Isocortex-un7: Isocortex, unassigned area 7')
in init_tool()
made it here
should be here
selecting region_map
layer name:
layer 1
selected values:
Map({"layer 1": {"value": {"key": "725", "value": "722", "label": "MH: Medial habenula"}}})
end of init_tool()
named tuple:
SegmentIdMapEntry(key=725, value=722, label='MH: Medial habenula')
in init_tool()
made it here
should be here
selecting region_map
layer name:
layer 1
selected values:
Map({"layer 1": {"value": {"key": "725", "value": "721", "label": "EPI: Epithalamus"}}})
end of init_tool()
named tuple:
SegmentIdMapEntry(key=725, value=721, label='EPI: Epithalamus')
in init_tool()
made it here
should be here
selecting region_map
layer name:
layer 1
selected valu

Traceback (most recent call last):
  File "/home/ahoag/anaconda3/envs/ng/lib/python3.6/site-packages/neuroglancer/viewer_config_state.py", line 113, in invoke
    handler(state)
  File "<ipython-input-23-9e9f34ee80dd>", line 51, in contract_atlas
    region_name = ontology_id_dict[region_id]
KeyError: 1346


in init_tool()
made it here
should be here
selecting region_map
layer name:
layer 1
selected values:
Map({"layer 1": {"value": {"key": "1346", "label": "root-un6: Unassigned area 6"}}})
end of init_tool()
named tuple:
SegmentIdMapEntry(key=1346, value=None, label='root-un6: Unassigned area 6')


Traceback (most recent call last):
  File "/home/ahoag/anaconda3/envs/ng/lib/python3.6/site-packages/neuroglancer/viewer_config_state.py", line 113, in invoke
    handler(state)
  File "<ipython-input-23-9e9f34ee80dd>", line 51, in contract_atlas
    region_name = ontology_id_dict[region_id]
KeyError: 1346


in init_tool()
made it here
should be here
selecting region_map
layer name:
layer 1
selected values:
Map({"layer 1": {"value": {"key": "1346", "label": "root-un6: Unassigned area 6"}}})
end of init_tool()
named tuple:
SegmentIdMapEntry(key=1346, value=None, label='root-un6: Unassigned area 6')


Traceback (most recent call last):
  File "/home/ahoag/anaconda3/envs/ng/lib/python3.6/site-packages/neuroglancer/viewer_config_state.py", line 113, in invoke
    handler(state)
  File "<ipython-input-23-9e9f34ee80dd>", line 51, in contract_atlas
    region_name = ontology_id_dict[region_id]
KeyError: 1346


in init_tool()
made it here
should be here
selecting region_map
layer name:
layer 1
selected values:
Map({"layer 1": {"value": {"key": "1346", "label": "root-un6: Unassigned area 6"}}})
end of init_tool()
named tuple:
SegmentIdMapEntry(key=1346, value=None, label='root-un6: Unassigned area 6')


Traceback (most recent call last):
  File "/home/ahoag/anaconda3/envs/ng/lib/python3.6/site-packages/neuroglancer/viewer_config_state.py", line 113, in invoke
    handler(state)
  File "<ipython-input-23-9e9f34ee80dd>", line 51, in contract_atlas
    region_name = ontology_id_dict[region_id]
KeyError: 1346


In [173]:
def reset_equivalences():
    with viewer.txn() as txn:
        txn.layers['Allen hierarch labels'].layer.equivalences = []


In [168]:
def make_subgraph(edge_list):
    subG = graphviz.Digraph()
    for edge in edge_list:
        if edge[0] == edge[1]:
            pass
        else:
            subG.edge(str(edge[0]),str(edge[1]))
    return subG